In [2]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]


In [2]:
startTm = datetime.datetime.now()

startDate = '20200722'
endDate = '20200724'

readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]
for thisDate in dateLs:    
    
    readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
    for col in ['clockAtArrival', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        orderLog[col] = orderLog[col].astype('int64')

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    orderLog = orderLog[orderLog["secid"] >= 1000000]

    orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    orderLog['broker'] = orderLog['accCode'] // 100
    orderLog['colo_account'] = orderLog['colo'].str[:2] + '_' + orderLog['accCode'].astype('str')
    orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
    orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
    orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
    orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
    orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
    orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
    display("Different orderDirection type:")
    display(orderLog["orderDirection"].unique())
    display(orderLog["orderDirection1"].unique())
    orderLog = orderLog.copy()

    ### make sure no order has shares > 80w or notional > 800w
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']

    ### make sure same direction in same colo_broker
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        orderLog = orderLog[orderLog['directNum'] == 1]

    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

    ## make sure each account, secid, vai only has one insertion
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        a = a[a>1].reset_index()
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]

    ### make sure there is no unexpected updateType 
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 2, 4, 1, 4, 3)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3)]), 2,
                     np.where(checkLog['updateType'].isin([(0, 3)]), 3,
                     np.where(checkLog['updateType'].isin([(0,), (0, 2), (0, 2, 1), (0, 2, 2), (0, 2, 4, 1)]), 4, 5)))))
    display(set(checkLog["updateType"].unique()) - set([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4),
                                                    (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4),
                                                    (0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3),
                                                    (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3),
                                                    (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 4, 2, 4, 1, 4, 3), (0, 2, 1, 3), (0, 2, 2, 1, 3),
                                                    (0, 3), (0,), (0, 2), (0, 2, 1), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), 
                                                    (0, 2, 2), (0, 2, 4, 1)]))

    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### check status==0 got all traded
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### check status==1 got partial traded
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### check if any cancellation within 1 sec
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'Different orderDirection type:'

array([ 1,  2, -1, -2,  0], dtype=int64)

array([ 1, -1,  0], dtype=int64)

{(1,), (7,)}

'Different orderDirection type:'

array([ 1, -1, -2,  2,  0], dtype=int64)

array([ 1, -1,  0], dtype=int64)

{(1,), (7,)}

In [15]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['secid'] < 2000000) & (orderLog['isMsg'] == 1)]

,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,tradeId,sdd,aaa,ApplSeqNum,clock,broker,colo_account,order,group,startClock,duration,orderDirection1,orderNtl,directNum,isMsg,status,exchange,tradeNtl
116,75076,1592967894559633,1592967894559289,1600021,0,2494935,331,0,-2,3600,0,7.49,-1.0,20200624,8971,100,zt_88_03,NaN,NaN,110430000,0.000679,1439763,2020-06-24 11:04:54.559633,89,zt_8971,7533,11,1592967894559633,0,-1,26964.0,1,1.0,0,SSE,0.0
120,17714,1592967872204920,1592967872204712,1600021,0,2494935,131,0,-2,3600,0,7.49,-1.0,20200624,9754,100,zs_96_08,NaN,NaN,110430000,0.000660,1439763,2020-06-24 11:04:32.204920,97,zs_9754,12592,11,1592967872204920,0,-1,26964.0,1,1.0,0,SSE,0.0
125,77036,1592967758981781,1592967758981689,1600025,0,23175836,131,0,1,33000,0,3.63,-1.0,20200624,537401,100,zt_52_07,NaN,NaN,110237000,0.001444,1656627,2020-06-24 11:02:38.981781,5374,zt_537401,17121,12,1592967758981781,0,1,119790.0,1,1.0,0,SSE,0.0
134,77057,1592977952269671,1592977952269537,1600025,0,34666720,131,0,1,1800,0,3.64,-1.0,20200624,537401,100,zt_52_07,NaN,NaN,135231000,0.000383,2343256,2020-06-24 13:52:32.269671,5374,zt_537401,17122,13,1592977952269671,0,1,6552.0,1,1.0,0,SSE,0.0
137,66737,1592977951962737,1592977951962684,1600025,0,34666720,11,0,1,129100,0,3.64,-1.0,20200624,966701,100,zt_96_04,NaN,NaN,135231000,0.000220,2343256,2020-06-24 13:52:31.962737,9667,zt_966701,17719,13,1592977951962737,0,1,469924.0,1,1.0,0,SSE,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31007,56931,1592977852511451,1592977852511243,1603987,0,18760687,131,0,-1,2600,0,16.79,-1.0,20200624,527601,100,zt_52_05,NaN,NaN,135043000,0.001078,2059876,2020-06-24 13:50:52.511451,5276,zt_527601,16598,4580,1592977852511451,0,-1,43654.0,1,1.0,1,SSE,0.0
31013,57048,1592978274765608,1592978274765494,1603987,0,19124447,131,0,-1,8400,0,16.70,-1.0,20200624,527601,100,zt_52_05,NaN,NaN,135746000,0.000784,2115347,2020-06-24 13:57:54.765608,5276,zt_527601,16599,4581,1592978274765608,0,-1,140280.0,1,1.0,0,SSE,0.0
31019,57118,1592978506267205,1592978506267094,1603987,0,19369247,131,0,-1,6000,0,16.69,-1.0,20200624,527601,100,zt_52_05,NaN,NaN,140138000,0.000922,2144004,2020-06-24 14:01:46.267205,5276,zt_527601,16600,4582,1592978506267205,0,-1,100140.0,1,1.0,0,SSE,0.0
31063,57464,1592978922521648,1592978922521486,1603987,0,19880067,131,0,-1,7500,0,16.83,-1.0,20200624,527601,100,zt_52_05,NaN,NaN,140835000,0.000904,2209712,2020-06-24 14:08:42.521648,5276,zt_527601,16604,4586,1592978922521648,0,-1,126225.0,1,1.0,0,SSE,0.0


In [11]:
kk[(kk['ApplSeqNum'] == 1439763) & (kk["SecurityID"] == 600021)]

,clockAtArrival,sequenceNo,exchId,securityType,__isRepeated,TransactTime,ChannelNo,ApplSeqNum,SecurityID,secid,mdSource,ExecType,TradeBSFlag,__origTickSeq,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
26138982,1592967875245024,59822472,1,1,0,110432060,2,1439763,600021,1600021,4,F,S,88784451,74900,1500,112350000,2530298,2531108


In [3]:
startTm = datetime.datetime.now()

startDate = '20200623'
endDate = '20200624'

readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

for thisDate in dateLs:    
    
    readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
    for col in ['clockAtArrival', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        orderLog[col] = orderLog[col].astype('int64')

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    orderLog = orderLog[orderLog["secid"] >= 1000000]

    orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    orderLog['broker'] = orderLog['accCode'] // 100
    orderLog['colo_account'] = orderLog['colo'].str[:2] + '_' + orderLog['accCode'].astype('str')
    orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
    orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
    orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
    orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
    orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
    orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
    display("Different orderDirection type:")
    display(orderLog["orderDirection"].unique())
    display(orderLog["orderDirection1"].unique())
    orderLog = orderLog.copy()

    ### make sure no order has shares > 80w or notional > 800w
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']

    ### make sure same direction in same colo_broker
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        orderLog = orderLog[orderLog['directNum'] == 1]

    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

    ## make sure each account, secid, vai only has one insertion
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        a = a[a>1].reset_index()
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]

    ### make sure there is no unexpected updateType 
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 2, 4, 1, 4, 3)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3)]), 2,
                     np.where(checkLog['updateType'].isin([(0, 3)]), 3,
                     np.where(checkLog['updateType'].isin([(0,), (0, 2), (0, 2, 1), (0, 2, 2), (0, 2, 4, 1)]), 4, 5)))))
    display(set(checkLog["updateType"].unique()) - set([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4),
                                                    (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4),
                                                    (0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3),
                                                    (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3),
                                                    (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 4, 2, 4, 1, 4, 3), (0, 2, 1, 3), (0, 2, 2, 1, 3),
                                                    (0, 3), (0,), (0, 2), (0, 2, 1), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), 
                                                    (0, 2, 2), (0, 2, 4, 1)]))

    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### check status==0 got all traded
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### check status==1 got partial traded
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### check if any cancellation within 1 sec
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)
    
    # 1. market orders
    readPath = r'L:\orderLog\mdData'
    rawMsgDataSZ = pd.read_pickle(os.path.join(readPath, 'mdLog_msg_%s.pkl'%thisDate))
    orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "agg", "TransactTime"]].reset_index(drop=True)
    orderDataSZ['updateType'] = 0
    tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                             rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
    tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
    tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
    tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
    tradeDataSZ['updateType'] = 4
    cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
    cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
    cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
    cancelDataSZ['updateType'] = 3

    msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
    del orderDataSZ
    del tradeDataSZ
    del cancelDataSZ
    msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
    msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
    msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
    msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
    msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
    msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
    assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
    msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
    msgDataSZ['date'] = int(thisDate) 
    msgDataSZ1 = msgDataSZ
    print('finish market orders')
 

    # 2. orderLog
#     startPos = startPos[((startPos.clock.dt.time >= datetime.time(9, 33)) & (startPos.clock.dt.time <= datetime.time(11, 30))) |\
#                         ((startPos.clock.dt.time >= datetime.time(13, 3)) & (startPos.clock.dt.time <= datetime.time(14, 55)))]
    infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog['isMsg'] == 1) 
                        & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
    infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
    infoData['Price'] = infoData['Price'].astype('int64')*100
    infoData['OrderQty'] = infoData['absOrderSize']
    infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
    infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
    infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
    statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
    statusInfo.columns = ['order', 'statusLs']
    tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['order', 'TradePriceLs']
    tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['order', 'TradeQtyLs']
    infoData = infoData[infoData['updateType'] == 0]
    infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
    infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])['colo_account'].transform('count')
    display(infoData[infoData['brokerNum'] >= 2].groupby(['colo', 'accCode'])['date'].count())
    display('%.2f%%'%(infoData[infoData['brokerNum'] >= 2].shape[0] / infoData.shape[0]*100))
    display(infoData[infoData['brokerNum'] >= 2].shape[0])
    display(infoData.shape[0])
    infoData = infoData[infoData['brokerNum'] == 1]
    infoData = infoData[['date', 'secid', 'vai', 'ars', 'exchange', 'group', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum', 'order', 'colo', 'accCode']]   
    
    print('finish our orders')
    
    # 3. find the position of our orders
    checkLog = msgDataSZ[msgDataSZ['updateType'] == 0]
    checkLog = pd.merge(checkLog, infoData.drop_duplicates(subset=['secid', 'ApplSeqNum'])[['secid', 'ApplSeqNum', 'ars']], 
                    on=['secid', 'ApplSeqNum'], how='left')
    checkLog['ApplSeqNum'] = np.where(~checkLog['ars'].isnull(), checkLog['ApplSeqNum'], checkLog['ars'])
    checkLog['start_time'] = np.where(~checkLog['ars'].isnull(), checkLog['clockAtArrival'], checkLog['ars'])
    checkLog.drop(["ars"],axis=1,inplace=True)
    checkLog['ApplSeqNum'] = checkLog.groupby(['secid'])['ApplSeqNum'].ffill()
    checkLog['start_time'] = checkLog.groupby(['secid'])['start_time'].ffill()
    checkLog['end_time'] = checkLog['start_time'] + 20*1e3
    checkLog = checkLog[(~checkLog['ApplSeqNum'].isnull()) & (checkLog['clockAtArrival'] > checkLog['start_time']) & 
                       (checkLog['clockAtArrival'] < checkLog['end_time'])]
    msgDataSZ = msgDataSZ[msgDataSZ['StockSeqNum'].isin(checkLog['StockSeqNum'].values)]
    print('finish get the interval')
    
    statusInfo = msgDataSZ.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    statusInfo.columns = ['StockSeqNum', 'statusLs']
    tradePriceInfo = msgDataSZ.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
    tradeQtyInfo = msgDataSZ.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
    checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')  

      
    try:
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], validate='many_to_one')
    except:
        print('There are orders with same pattern but different vai, same ApplSeqNum')
        display([infoData[infoData.duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]])
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])
    savePath = r'L:\orderLog\result\marketPos'
    checkLog.reset_index(drop=True).to_pickle(os.path.join(savePath, 'SZspeed_%s.pkl'%thisDate))
    
print(datetime.datetime.now() - startTm)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'Different orderDirection type:'

array([ 1,  2, -1, -2,  0], dtype=int64)

array([ 1, -1,  0], dtype=int64)

{(1,), (7,)}

...

colo      accCode
zs_52_02  5276        63
          5277        79
zs_52_06  5273       147
          5275       130
zs_54_01  5456        71
          5470        37
          5474       141
zs_64_01  6479        19
zs_66_01  6678        15
zs_88_01  8856         6
          8865        45
          8867        60
          885602      12
zs_92_01  9208        61
zs_92_02  9243        40
zs_94_03  9448        39
          9451        46
          9461        12
zs_94_04  6237        63
zs_94_05  9454        19
          9471        91
zs_96_06  9756        76
          9765        60
zs_96_08  6272        72
          9754        63
zt_88_02  8854        13
zt_88_03  8970        13
          8971        10
zt_96_01  966301      20
Name: date, dtype: int64

'13.60%'

1523

11199

finish our orders
finish get the interval
0:08:49.005334


In [8]:
orderLog[orderLog['group'].isin([13167, 13168])]

,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,tradeId,sdd,aaa,ApplSeqNum,clock,broker,colo_account,order,group,startClock,duration,orderDirection1,orderNtl,directNum,isMsg,status,exchange,tradeNtl
41195,24349,1592200346955341,1592200346955319,2002947,0,1528599,11,0,1,100,0,54.2,-1.0,20200615,5456,100,zs_54_01,NaN,NaN,135220420,0.001932,15113552,2020-06-15 13:52:26.955341,54,zs_5456,2897,13167,1592200346955341,0,1,5420.0,1,1.0,0,SZE,0.0
41196,24350,1592200346958059,1592200346955319,2002947,2,1528599,-1,0,1,100,0,54.2,-1.0,20200615,5456,0,zs_54_01,00C0QBXVGD01D2LF,NaN,-1,-1.000000,0,2020-06-15 13:52:26.958059,54,zs_5456,2897,13167,1592200346955341,2718,1,5420.0,1,1.0,0,SZE,0.0
41197,24351,1592200346958124,1592200346955319,2002947,4,1528599,-1,100,1,100,100,54.2,54.2,20200615,5456,0,zs_54_01,NaN,1.04e+14,-1,-1.000000,0,2020-06-15 13:52:26.958124,54,zs_5456,2897,13167,1592200346955341,2783,1,5420.0,1,1.0,0,SZE,5420.0
41198,77667,1592200340430022,1592200340429953,2002947,0,1528799,11,0,1,100,0,54.2,-1.0,20200615,5275,100,zs_52_02,NaN,NaN,135220420,0.001897,15113552,2020-06-15 13:52:20.430022,52,zs_5275,1737,13168,1592200340430022,0,1,5420.0,1,1.0,0,SZE,0.0
41199,77668,1592200340433138,1592200340432734,2002947,2,1528799,-1,0,1,100,0,54.2,-1.0,20200615,5275,0,zs_52_02,241123.0,NaN,-1,-1.000000,0,2020-06-15 13:52:20.433138,52,zs_5275,1737,13168,1592200340430022,3116,1,5420.0,1,1.0,0,SZE,0.0
41200,77669,1592200340433230,1592200340432734,2002947,4,1528799,-1,100,1,100,100,54.2,54.2,20200615,5275,0,zs_52_02,NaN,1.04e+14,-1,-1.000000,0,2020-06-15 13:52:20.433230,52,zs_5275,1737,13168,1592200340430022,3208,1,5420.0,1,1.0,0,SZE,5420.0


In [118]:
checkLog[~checkLog['ars'].isnull()].drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep='last')

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode
331,1,374044.0,1591579804519226,2555100,2.0,8400.0,136800.0,929334.0,0.0,93001430,0,0,0,2000001,1.004266e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4, 4)","(0, 136800, 136800, 136800)","(0, 800, 1900, 5700)",735734.0,11.0,SZE,4508.0,13593.0,zs_96_06,9765.0
844,1,515330.0,1591579808376541,3095238,2.0,5500.0,136800.0,1086759.0,0.0,93005820,0,0,0,2000001,1.005506e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 136800)","(0, 5500)",1040659.0,11.0,SZE,4509.0,13594.0,zs_96_06,9765.0
953,1,566081.0,1591579809183945,3211474,2.0,800.0,136700.0,1152259.0,1.0,93007400,0,0,0,2000001,1.005767e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 136800)","(0, 800)",1101959.0,11.0,SZE,4510.0,13595.0,zs_96_06,9765.0
979,1,583431.0,1591579809414465,3240320,2.0,3200.0,136800.0,1169159.0,1.0,93007950,0,0,0,2000001,1.005835e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4)","(0, 136800, 136800)","(0, 2900, 300)",1165959.0,11.0,SZE,4511.0,13596.0,zs_96_06,9765.0
1008,1,592180.0,1591579810167174,3284621,2.0,900.0,136900.0,1335559.0,1.0,93008700,0,0,0,2000001,1.005922e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 136900)","(0, 900)",1334659.0,11.0,SZE,4512.0,13597.0,zs_96_06,9765.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096959,300788,18156572.0,1591599204437184,112228391,2.0,100.0,452100.0,411900.0,0.0,145322920,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452100)","(0, 100)",410900.0,11.0,SZE,12667.0,2368.0,zs_52_02,5275.0
1096986,300788,18218792.0,1591599234894104,112529190,2.0,100.0,452600.0,414900.0,1.0,145353380,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452600)","(0, 100)",414800.0,11.0,SZE,12669.0,2370.0,zs_52_02,5275.0
1096991,300788,18233512.0,1591599243238617,112613792,2.0,100.0,452400.0,415300.0,1.0,145401720,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452400)","(0, 100)",415200.0,11.0,SZE,12670.0,2371.0,zs_52_02,5275.0
1097013,300788,18266606.0,1591599260383676,112798687,2.0,100.0,452400.0,417000.0,1.0,145418870,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452400)","(0, 100)",416900.0,11.0,SZE,12671.0,2372.0,zs_52_02,5275.0


In [50]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['isMsg'] == 1)].groupby(['colo', 'accCode'])['ars'].count()

colo      accCode
zs_52_02  5276        308
          5277        346
zs_52_06  5273        946
          5275       1039
zs_54_01  5456        421
          5470        116
          5474        607
zs_64_01  6479        156
zs_66_01  6678        105
zs_88_01  8856         26
          8865        463
          8867        392
          885602       98
zs_92_01  9208        294
zs_92_02  9243        312
zs_94_03  9448        226
          9451        403
          9461         85
zs_94_04  6237        196
zs_94_05  9454        616
          9471        989
zs_96_06  9756        574
          9765        316
zs_96_08  6272        242
          9754        458
zt_88_02  8854        329
zt_88_03  8970        106
          8971         75
zt_96_01  966301      180
Name: ars, dtype: int64

In [59]:
infoData['time'] = infoData['clock'].apply(lambda x: x.strftime("%H%M%S")).astype(int)
infoData[infoData['accCode'].isin([6272, 6237])]['time'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count       438.000000
mean     116806.593607
std       20910.962290
min       93000.000000
10%       93131.700000
20%       93932.400000
30%       95802.400000
40%      105234.000000
50%      110321.500000
60%      130435.400000
70%      135836.300000
80%      140307.000000
90%      142291.400000
max      145739.000000
Name: time, dtype: float64

In [48]:
orderLog[(orderLog['isMsg']==1) & (orderLog['accCode']==6272)].groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index().groupby(['updateType'])['order'].count()

updateType
(0, 2, 1, 3)                                                             68
(0, 2, 1, 4, 3)                                                           1
(0, 2, 1, 4, 4)                                                           1
(0, 2, 4)                                                                91
(0, 2, 4, 1, 3)                                                          14
(0, 2, 4, 1, 4, 3)                                                        1
(0, 2, 4, 4)                                                             28
(0, 2, 4, 4, 1, 3)                                                        4
(0, 2, 4, 4, 4)                                                           9
(0, 2, 4, 4, 4, 1, 3)                                                     1
(0, 2, 4, 4, 4, 4)                                                        6
(0, 2, 4, 4, 4, 4, 1, 3)                                                  2
(0, 2, 4, 4, 4, 4, 4)                                                     3
(

In [101]:
orderLog[(orderLog['secid'] == 2002434) & (orderLog['accCode'] == 6272)]

,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,tradeId,sdd,aaa,ApplSeqNum,clock,broker,colo_account,order,group,startClock,duration,orderDirection1,orderNtl,directNum,isMsg,status,exchange,tradeNtl
16135,31832,1593399172478365,1593399172478270,2002434,0,2897502,131,0,-1,700,0,8.7,-1.0,20200629,6272,100,zs_96_08,NaN,NaN,105252500,0.000508,9099215,2020-06-29 10:52:52.478365,62,zs_6272,4526,6970,1593399172478365,0,-1,6090.0,1,1.0,0,SZE,0.0
16136,31833,1593399172509048,1593399172478270,2002434,2,2897502,-1,0,-1,700,0,8.7,-1.0,20200629,6272,0,zs_96_08,1593391484265464.0,NaN,-1,-1.000000,0,2020-06-29 10:52:52.509048,62,zs_6272,4526,6970,1593399172478365,30683,-1,6090.0,1,1.0,0,SZE,0.0
16137,31834,1593399172568072,1593399172510841,2002434,4,2897502,-1,700,-1,700,700,8.7,8.7,20200629,6272,0,zs_96_08,NaN,,-1,-1.000000,0,2020-06-29 10:52:52.568072,62,zs_6272,4526,6970,1593399172478365,89707,-1,6090.0,1,1.0,0,SZE,6090.0


In [103]:
msgDataSZ1[(msgDataSZ1["secid"] == 2002434) & (msgDataSZ1["ApplSeqNum"] >= 9099215) ].sort_values(by='sequenceNo').head(20)

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date
25909527,2434,9099215.0,1593399176502766,55034797,1.0,2600.0,87000.0,2897502.0,1.0,105252500,0,0,0,2002434,2.434091e+11,20200629
25909528,2434,9099215.0,1593399176502769,55034798,1.0,NaN,NaN,2897502.0,NaN,105252500,4,87000,1900,2002434,2.434091e+11,20200629
25909530,2434,9099225.0,1593399176519315,55034869,1.0,100.0,87000.0,2897502.0,0.0,105252520,0,0,0,2002434,2.434091e+11,20200629
25909532,2434,9099228.0,1593399176535319,55034892,2.0,700.0,87000.0,2898202.0,1.0,105252540,0,0,0,2002434,2.434091e+11,20200629
25909529,2434,9099215.0,1593399176535322,55034893,1.0,NaN,NaN,2898202.0,NaN,105252540,4,87000,700,2002434,2.434091e+11,20200629
25909533,2434,9099228.0,1593399176535322,55034893,2.0,NaN,NaN,2898202.0,NaN,105252540,4,87000,700,2002434,2.434091e+11,20200629
25909534,2434,9099230.0,1593399176539195,55034908,2.0,100.0,87100.0,2898202.0,0.0,105252540,0,0,0,2002434,2.434091e+11,20200629
25909536,2434,9100134.0,1593399177899324,55041155,2.0,2800.0,86800.0,2901002.0,1.0,105253900,0,0,0,2002434,2.434091e+11,20200629
25909531,2434,9099225.0,1593399177899328,55041156,1.0,NaN,NaN,2901002.0,NaN,105253900,4,87000,100,2002434,2.434091e+11,20200629
25909537,2434,9100134.0,1593399177899328,55041156,2.0,NaN,NaN,2901002.0,NaN,105253900,4,87000,100,2002434,2.434091e+11,20200629


In [91]:
infoData[(infoData['secid'] == 2002127) & (infoData['ApplSeqNum'] == 9559310)]

,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,tradeId,sdd,aaa,ApplSeqNum,clock,broker,colo_account,order,group,startClock,duration,orderDirection1,orderNtl,directNum,isMsg,status,exchange,tradeNtl,Price,OrderQty,Side,TradePrice,statusLs,TradePriceLs,TradeQtyLs,brokerNum,time
2388,24716,1593399089336291,1593399089336281,2002127,0,12713009,11,0,-1,200,0,21.44,-1.0,20200629,6237,100,zs_94_04,NaN,NaN,105004370,0.00054,9559310,2020-06-29 10:51:29.336291,62,zs_6237,4105,6163,1593399089336291,0,-1,4288.0,1,1.0,0,SZE,0.0,214400,200,2,0,"(0, 4)","(0, 214400)","(0, 200)",1,105129


In [93]:
checkLog = checkLog[~checkLog['ars'].isnull()]
checkLog[checkLog['accCode'] == 6237]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode
570,501,13717155.0,1593410794201409,91273451,2.0,200.0,144500.0,15470191.0,1.0,140630180,0,0,0,2000501,5.011372e+10,20200629,1.593411e+15,1.593411e+15,"(0, 4)","(0, 144500)","(0, 200)",15469391.0,11.0,SZE,4771.0,4058.0,zs_94_04,6237.0
4023,2242,10968950.0,1593406906862741,69140891,1.0,900.0,368600.0,1638956.0,0.0,130142850,0,0,0,2002242,2.242110e+11,20200629,1.593407e+15,1.593407e+15,"(0, 3)","(0, 0)","(0, 0)",1638056.0,11.0,SZE,6461.0,4122.0,zs_94_04,6237.0
4033,2242,10983227.0,1593406923550611,69238886,1.0,1700.0,367000.0,1654456.0,0.0,130159530,0,0,0,2002242,2.242110e+11,20200629,1.593407e+15,1.593407e+15,"(0, 3)","(0, 0)","(0, 0)",1652756.0,11.0,SZE,6462.0,4123.0,zs_94_04,6237.0
4043,2242,10983227.0,1593406923550645,69238896,1.0,1700.0,367000.0,1654456.0,0.0,130159530,0,0,0,2002242,2.242110e+11,20200629,1.593407e+15,1.593407e+15,"(0, 3)","(0, 0)","(0, 0)",1652756.0,11.0,SZE,6462.0,4123.0,zs_94_04,6237.0
5411,2414,10216266.0,1593399755963635,58179027,2.0,100.0,283600.0,23120084.0,1.0,110231960,0,0,0,2002414,2.414102e+11,20200629,1.593400e+15,1.593400e+15,"(0, 4)","(0, 283600)","(0, 100)",23119984.0,11.0,SZE,6897.0,4149.0,zs_94_04,6237.0


In [145]:
test = checkLog[~checkLog['ars'].isnull()]
test = test.reset_index(drop=True)
test['time_diff'] = test['clockAtArrival'] - test['start_time']
test['index'] = test.index.values
test['lastIx'] = test.groupby(['date','SecurityID', 'ApplSeqNum', 'colo','accCode'])['index'].transform('last')
test = test[test['index'] == test['lastIx']]

In [144]:
test.drop_duplicates(['date','ApplSeqNum', 'colo','accCode','Price'])[
    test.drop_duplicates(['date','ApplSeqNum', 'colo','accCode','Price']).duplicated(['date','ApplSeqNum', 'colo','accCode'],keep=False)]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode
152756,799,744272.0,1591579832735615,3945112,2.0,2700.0,513800.0,536421.0,1.0,93031270,0,0,0,2000799,7.990074e+10,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4, 4, 4, 4, 4)","(0, 514100, 514000, 514000, 514000, 514000, 51...","(0, 479, 500, 200, 500, 100, 921)",533721.0,11.0,SZE,5381.0,12938.0,zs_96_06,9758.0
869795,300299,611753.0,1591579809875075,3270016,2.0,5300.0,61800.0,2150600.0,1.0,93008410,0,0,0,2300299,3.002990e+13,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4)","(0, 61900, 61800)","(0, 5000, 300)",2145300.0,11.0,SZE,10352.0,10815.0,zs_94_05,9471.0
883631,300327,744272.0,1591579833724562,3972633,2.0,100.0,321400.0,111057.0,1.0,93032260,0,0,0,2300327,3.003270e+13,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 321400)","(0, 100)",110957.0,11.0,SZE,10519.0,13337.0,zs_96_06,9758.0
930615,300429,611753.0,1591579812551720,3387765,2.0,28200.0,193500.0,707920.0,1.0,93011090,0,0,0,2300429,3.004290e+13,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4, 4, 4)","(0, 193600, 193600, 193500, 193500)","(0, 26900, 200, 680, 420)",679720.0,11.0,SZE,10902.0,10899.0,zs_94_05,9471.0


In [146]:
test

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode,time_diff,index,lastIx
0,1,374044.0,1591579804519226,2555100,2.0,8400.0,136800.0,929334.0,0.0,93001430,0,0,0,2000001,1.004266e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4, 4)","(0, 136800, 136800, 136800)","(0, 800, 1900, 5700)",735734.0,11.0,SZE,4508.0,13593.0,zs_96_06,9765.0,1627864.0,0,0
1,1,515330.0,1591579808376541,3095238,2.0,5500.0,136800.0,1086759.0,0.0,93005820,0,0,0,2000001,1.005506e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 136800)","(0, 5500)",1040659.0,11.0,SZE,4509.0,13594.0,zs_96_06,9765.0,1094535.0,1,1
2,1,566081.0,1591579809183945,3211474,2.0,800.0,136700.0,1152259.0,1.0,93007400,0,0,0,2000001,1.005767e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 136800)","(0, 800)",1101959.0,11.0,SZE,4510.0,13595.0,zs_96_06,9765.0,325496.0,2,2
3,1,583431.0,1591579809414465,3240320,2.0,3200.0,136800.0,1169159.0,1.0,93007950,0,0,0,2000001,1.005835e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4, 4)","(0, 136800, 136800)","(0, 2900, 300)",1165959.0,11.0,SZE,4511.0,13596.0,zs_96_06,9765.0,3936.0,3,3
4,1,592180.0,1591579810167174,3284621,2.0,900.0,136900.0,1335559.0,1.0,93008700,0,0,0,2000001,1.005922e+08,20200608,1.591580e+15,1.591580e+15,"(0, 4)","(0, 136900)","(0, 900)",1334659.0,11.0,SZE,4512.0,13597.0,zs_96_06,9765.0,3841.0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,300788,18156572.0,1591599204437184,112228391,2.0,100.0,452100.0,411900.0,0.0,145322920,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452100)","(0, 100)",410900.0,11.0,SZE,12667.0,2368.0,zs_52_02,5275.0,6834809.0,9631,9631
9632,300788,18218792.0,1591599234894104,112529190,2.0,100.0,452600.0,414900.0,1.0,145353380,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452600)","(0, 100)",414800.0,11.0,SZE,12669.0,2370.0,zs_52_02,5275.0,4339.0,9632,9632
9633,300788,18233512.0,1591599243238617,112613792,2.0,100.0,452400.0,415300.0,1.0,145401720,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452400)","(0, 100)",415200.0,11.0,SZE,12670.0,2371.0,zs_52_02,5275.0,3751.0,9633,9633
9634,300788,18266606.0,1591599260383676,112798687,2.0,100.0,452400.0,417000.0,1.0,145418870,0,0,0,2300788,3.007882e+13,20200608,1.591599e+15,1.591599e+15,"(0, 4)","(0, 452400)","(0, 100)",416900.0,11.0,SZE,12671.0,2372.0,zs_52_02,5275.0,3880.0,9634,9634


In [2]:
startDate = '20200623'
endDate = '20200701'
readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'SZspeed_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    data = pd.read_pickle(path)
    checkData += [data]
checkLog = pd.concat(checkData).reset_index(drop=True)   
checkLog['date'].unique()

array([20200701, 20200629, 20200630, 20200623, 20200624], dtype=int64)

In [4]:
# thisDate = 20200701
# checkLog = pd.read_pickle(os.path.join(savePath, 'SZspeed_%s.pkl'%thisDate))
checkLog = checkLog[~checkLog['ars'].isnull()]
display(checkLog.shape[0])
display(checkLog[checkLog.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep='last')].shape[0])
display(checkLog[checkLog.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)].shape[0])
display(checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep='last').shape[0])
display(checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False).shape[0])
checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog['time_diff'] = checkLog['clockAtArrival'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
# # staone orders
from IPython.display import display, HTML
re1 = checkLog[~checkLog['ars'].isin([121, 221, 321, 131, 231, 331]) & (checkLog['colo1'] != 'zt')].groupby(['colo', 'accCode'])['time_diff'].describe().fillna(0).astype(int)
re1 = re1[re1['count'] > 20].reset_index()
re1['accCode'] = re1['accCode'].astype(int)
display(HTML(re1[['colo', 'accCode', 'count', '50%']].sort_values(by='50%').to_html()))
# # statow orders
# display(HTML(checkLog[checkLog['ars'].isin([121, 221, 321, 131, 231, 331]) & (checkLog['colo1'] != 'zt')].groupby(['colo', 'accCode'])['time_diff'].describe().fillna(0).astype(int).to_html()))

34550

0

0

34550

34550

,colo,accCode,count,50%
23,zs_96_06,9758,152,3188
19,zs_94_03,9461,381,3532
25,zs_96_08,9754,268,3555
18,zs_94_03,9451,1802,3573
24,zs_96_06,9765,181,3583
5,zs_52_06,5275,1219,3590
4,zs_52_06,5273,1236,3592
17,zs_94_03,9448,562,3605
8,zs_54_01,5474,2391,3633
21,zs_94_05,9471,385,3634


In [28]:
re1

count  mean   std   min   25%   50%   75%    max
colo     accCode                                                   
zs_52_02 5276.0      584  3604   466  2489  3400  3721  3894   5108
         5277.0      670  4402   982  2749  3964  4322  4608  15320
zs_52_06 5273.0      891  3674   808   412  3521  3625  3735  14340
         5275.0      903  3627   298  2401  3524  3616  3714   6353
zs_54_01 5456.0      687  3679   626  2183  3446  3702  3955   7185
         5470.0      201  3838   989  2399  3424  3728  4073  11241
         5474.0     1332  3666   912  2261  3297  3653  3936  15433
zs_64_01 6479.0      168  5196   605  3971  4842  5228  5449   7823
zs_66_01 6678.0      115  7157  5000  3852  4226  5141  5457  19810
zs_88_01 8856.0       76  3977  1020  2949  3444  3905  4261  11657
         8865.0      864  4801  1178  3194  4274  4803  5064  19284
         8867.0      804  4912  1361  3376  4487  4826  5097  18753
         885602.0    146  3939   720  2812  3501  3953  4305  10067
zs_92_01 9208.0      558  3713   692  2638  3463  3754  3959  14195
zs_92_02 9243.0      460  3772   847  2630  3528  3784  3979  14530
zs_94_03 9448.0      365  4203  2816  2376  3259  3631  4054  19146
         9451.0     1089  3700  1625  2305  3217  3596  3821  19775
         9461.0      198  4106  2954  2327  2890  3543  3820  18946
zs_94_04 6237.0        3  3783   296  3557  3616  3675  3897   4119
zs_94_05 9454.0      120  5795  4582   400  3476  3671  4519  19129
         9471.0      244  5268  4079  2486  3516  3668  4300  19829
zs_96_06 9756.0      194  3748   517  2561  3564  3697  3867   6368
         9758.0      152  3259   570    36  2880  3188  3675   5098
         9765.0       93  3699   331  3355  3538  3605  3719   5692
zs_96_08 9754.0       89  3763   528  2583  3514  3789  3989   5177

In [26]:
checkLog[checkLog.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)].groupby(['colo', 'accCode'])['order'].size()

colo      accCode 
zs_52_02  5276.0       18
          5277.0       26
zs_52_06  5273.0      136
          5275.0       93
zs_54_01  5456.0       28
          5470.0       24
          5474.0       45
zs_64_01  6479.0       32
zs_66_01  6678.0       71
zs_88_01  8856.0       17
          8865.0       39
          8867.0      110
          885602.0     83
zs_92_01  9208.0       33
zs_92_02  9243.0       13
zs_94_03  9448.0       14
          9451.0       29
          9461.0        8
zs_94_04  6237.0        2
zs_94_05  9454.0       24
          9471.0       67
zs_96_06  9756.0       93
          9758.0       25
          9765.0       39
zs_96_08  9754.0       78
zt_88_02  8854.0       63
zt_88_03  8970.0        4
          8971.0       21
zt_96_01  966301.0     43
Name: order, dtype: int64

In [12]:
re1

,colo,accCode,count,mean,std,min,25%,50%,75%,max
0,zs_52_02,5276,584,3604,466,2489,3400,3721,3894,5108
1,zs_52_02,5277,670,4402,982,2749,3964,4322,4608,15320
2,zs_52_06,5273,891,3674,808,412,3521,3625,3735,14340
3,zs_52_06,5275,903,3627,298,2401,3524,3616,3714,6353
4,zs_54_01,5456,687,3679,626,2183,3446,3702,3955,7185
5,zs_54_01,5470,201,3838,989,2399,3424,3728,4073,11241
6,zs_54_01,5474,1332,3666,912,2261,3297,3653,3936,15433
7,zs_64_01,6479,168,5196,605,3971,4842,5228,5449,7823
8,zs_66_01,6678,115,7157,5000,3852,4226,5141,5457,19810
9,zs_88_01,8856,76,3977,1020,2949,3444,3905,4261,11657


In [50]:
checklog['broker'] = checkLog['colo'].str[:2] + '_' + checkLog["accCode"].astype(str).str[:2]

0        zs_54
1        zs_94
2        zs_94
3        zs_94
6        zs_52
         ...  
17170    zs_88
17175    zs_92
17176    zs_88
17177    zs_94
17186    zs_94
Length: 6516, dtype: object

In [25]:
checkLog[~checkLog['ars'].isin([121, 221, 321, 131, 231, 331])]['colo'].unique()

array(['zs_96_06', 'zs_54_01', 'zs_52_02', 'zs_96_08', 'zs_94_03',
       'zs_92_02', 'zs_88_01', 'zs_94_05', 'zs_92_01', 'zt_88_02',
       'zs_94_04'], dtype=object)

In [160]:
checkData = checkData[~checkData['ars'].isnull()]
checkData = checkData.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkData['time_diff'] = checkData['clockAtArrival'] - checkData['start_time']
# staone orders
display(checkData[~checkData['ars'].isin([121, 221, 321, 131, 231, 331])].groupby(['colo', 'accCode'])['time_diff'].describe().astype('int64'))
# statow orders
display(checkData[checkData['ars'].isin([121, 221, 321, 131, 231, 331])].groupby(['colo', 'accCode'])['time_diff'].describe().astype('int64'))

count     mean       std    min     25%     50%      75%  \
colo     accCode                                                             
zs_52_02 5273.0     837  1754860   9563814   2558    2974    3819     4067   
         5275.0     909  1061001   7566959   2090    3574    3817     4060   
         5277.0     210  1376501   6444117   2637    3643    3878    12244   
zs_54_01 5456.0     246  1786799   8772202   2481    3469    3764     4153   
         5470.0     462  1381967   7558748   1218    3405    3823     4807   
         5474.0     613  1412765   8218832   1216    3000    3726     4132   
zs_88_01 8856.0      47  1887185   7003825   2806    3557    3970     4737   
         8865.0     364  3024097  12983017   3400    4466    4797     5307   
         8867.0     303  3523614  15109158   3423    4517    4861     5243   
zs_92_01 9208.0     173  1843398   9002011   1481    3136    3788     4791   
zs_92_02 9243.0     160  1820623   8297748   2704    3459    3804     4719   
zs_94_03 9448.0     143  1775643   9353993   2427    3457    3748    21664   
         9451.0     386  2095241  10515742   2420    3466    3697     4521   
         9461.0      86  1241357  10630114   2498    3258    3624     3826   
zs_94_04 6237.0      33  5818560  15350518  30514   31800   35156   385757   
zs_94_05 9454.0     115   325196    896260   1230    3650    4254    24438   
         9471.0     465  1417117   7249618   2494    3618    4421    59560   
zs_96_06 9756.0     143  1472080   9226303   2541    3614    3730     3962   
         9758.0     548  2709112  12534371   1659    3609    3812     4083   
         9765.0     110  2566027  11175169   3368    3671    3847  1028745   
zs_96_08 6272.0      16  5827468  21259993  41918  247054  413284   795746   
         9754.0     114  4448962  14832952   2702    4064    4394  1293366   
zt_88_02 8854.0     248  6172641  18861594  34686   35930   36953   492425   

                       max  
colo     accCode            
zs_52_02 5273.0   97958396  
         5275.0   94029653  
         5277.0   53868145  
zs_54_01 5456.0   81003736  
         5470.0   92584174  
         5474.0   98800597  
zs_88_01 8856.0   46045568  
         8865.0   98999961  
         8867.0   98029462  
zs_92_01 9208.0   92632367  
zs_92_02 9243.0   75239894  
zs_94_03 9448.0   90509761  
         9451.0   98907658  
         9461.0   98613157  
zs_94_04 6237.0   60913917  
zs_94_05 9454.0    7981973  
         9471.0   84648221  
zs_96_06 9756.0   99213695  
         9758.0   91032099  
         9765.0   86630108  
zs_96_08 6272.0   85538608  
         9754.0   92178769  
zt_88_02 8854.0   98148384

count     mean       std    min    25%    50%    75%  \
colo     accCode                                                         
zs_94_05 9454.0     235  1547412   8988576   2543   3113   3774   3952   
         9471.0     466  1031149   7493490   2404   3123   3746   3895   
zs_96_06 9756.0     273  1324061   7865318   2399   3553   3860   4075   
         9765.0     200  4705630  17194622   2541   3684   3842   4038   
zs_96_08 6272.0      12  1997627   6684940  30332  32180  43016  69758   
         9754.0     193  2980870  13597367   2857   3377   4217   5456   

                       max  
colo     accCode            
zs_94_05 9454.0   74733856  
         9471.0   95806090  
zs_96_06 9756.0   87254692  
         9765.0   96238848  
zs_96_08 6272.0   23223908  
         9754.0   99249223

In [3]:
dateLs = ['20200722', '20200723', '20200724']

for date in dateLs:
    orderLog = pd.read_pickle(r'A:\temp\speedCompare_6237_6282.pkl')  
    targetStockSZ = sorted(orderLog[(orderLog['secid'] > 2000000) & (orderLog['date'] == int(date))]['secid'].astype(int).unique() - 2000000)

    readPath = r'\\192.168.10.30\Kevin_zhenyu\rawData'
    mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
    mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]
    
    mdOrderLog = pd.read_csv(mdOrderLogPath)
    mdOrderLog = mdOrderLog[mdOrderLog['SecurityID'].isin(targetStockSZ)]
    mdOrderLog['OrderType'] = mdOrderLog['OrderType'].astype(str)
    mdOrderLog = mdOrderLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                             'OrderType', 'Price', 'OrderQty']]
    
    mdTradeLog = pd.read_csv(mdTradeLogPath)
    mdTradeLog['ExecType'] = mdTradeLog['ExecType'].astype(str)
    mdTradeLog = mdTradeLog[mdTradeLog['SecurityID'].isin(targetStockSZ)]
    mdTradeLog['volumeThisUpdate'] = np.where(mdTradeLog['ExecType'] == 'F', mdTradeLog['TradeQty'], 0)
    mdTradeLog['cum_volume'] = mdTradeLog.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
    mdTradeLog = mdTradeLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                             'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]

    mdMsgData = pd.concat([mdOrderLog, mdTradeLog], sort=False)
    del mdOrderLog
    del mdTradeLog
    
    mdMsgData = mdMsgData.sort_values(by=['sequenceNo']).reset_index(drop=True)
    
    mdMsgData["agg_trade"] = np.where((mdMsgData["ApplSeqNum"] == mdMsgData["BidApplSeqNum"] + 1) & (mdMsgData["ExecType"] == "F"), 1, np.where(
    (mdMsgData["ApplSeqNum"] == mdMsgData["OfferApplSeqNum"] + 1) & (mdMsgData["ExecType"] == "F"), 1, 0))
    mdMsgData["agg"] = mdMsgData.groupby(["SecurityID"])["agg_trade"].shift(-1)
    mdMsgData["orderNum"] = np.where(mdMsgData["ExecType"].isnull(), 1, 0)
    mdMsgData["cumorderNum"] = mdMsgData.groupby("SecurityID")["orderNum"].cumsum()
    mdMsgData["cumorderNum2"] = np.nan
    mdMsgData.loc[mdMsgData["agg"]==1, "cumorderNum2"] = mdMsgData.loc[mdMsgData["agg"]==1, "cumorderNum"]
    mdMsgData["cumorderNum2"] = mdMsgData.groupby("SecurityID")["cumorderNum2"].ffill()
    mdMsgData.loc[mdMsgData["cumorderNum2"] == mdMsgData["cumorderNum"], "cum_volume"] = mdMsgData[mdMsgData["cumorderNum2"] == mdMsgData["cumorderNum"]]\
    .groupby(["SecurityID", "cumorderNum"])["cum_volume"].transform("max")
    
    mdMsgData = mdMsgData.sort_values(by=['sequenceNo']).reset_index(drop=True)

    mdMsgData['cum_volume'] = mdMsgData.groupby(['SecurityID'])['cum_volume'].ffill()
    mdMsgData['cum_volume'] = mdMsgData.groupby(['SecurityID'])['cum_volume'].backfill()
    mdMsgData['ExecType'] = mdMsgData['ExecType'].fillna('2')
    mdMsgData['TradeQty'] = mdMsgData['TradeQty'].fillna(0)
    
    saveCols = ['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'cum_volume', 'ApplSeqNum', 
                'Side', 'OrderType', 'Price', 'OrderQty', 'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney',
                'BidApplSeqNum', 'OfferApplSeqNum', "agg"]
    mdMsgData = mdMsgData[saveCols]
    savePath = r'L:\orderLog\mdData'
    mdMsgData.to_pickle(os.path.join(savePath, 'mdLog_msg_%s.pkl'%date))
    

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Ana

NameError: name 'startTm' is not defined

In [36]:
dateLs = ['20200722', '20200723', '20200724']

for thisDate in dateLs:    
    
    orderLog = pd.read_pickle(r'A:\temp\speedCompare_6237_6282.pkl')  
    orderLog = orderLog[orderLog['date'] == int(thisDate)]

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    orderLog = orderLog[orderLog["secid"] >= 1000000]

    orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    orderLog['broker'] = orderLog['accCode'] // 100
    orderLog['colo_account'] = orderLog['colo'].str[:2] + '_' + orderLog['accCode'].astype('str')
    orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
    orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
    orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
    orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
    orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
    orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
    display("Different orderDirection type:")
    display(orderLog["orderDirection"].unique())
    display(orderLog["orderDirection1"].unique())
    orderLog = orderLog.copy()

    ### make sure no order has shares > 80w or notional > 800w
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']

    ### make sure same direction in same colo_broker
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        orderLog = orderLog[orderLog['directNum'] == 1]

    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

    ## make sure each account, secid, vai only has one insertion
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        a = a[a>1].reset_index()
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]

    ### make sure there is no unexpected updateType 
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 2, 4, 1, 4, 3)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3)]), 2,
                     np.where(checkLog['updateType'].isin([(0, 3)]), 3,
                     np.where(checkLog['updateType'].isin([(0,), (0, 2), (0, 2, 1), (0, 2, 2), (0, 2, 4, 1)]), 4, 5)))))
    display(set(checkLog["updateType"].unique()) - set([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4),
                                                    (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4),
                                                    (0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3),
                                                    (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3),
                                                    (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 4, 2, 4, 1, 4, 3), (0, 2, 1, 3), (0, 2, 2, 1, 3),
                                                    (0, 3), (0,), (0, 2), (0, 2, 1), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), 
                                                    (0, 2, 2), (0, 2, 4, 1)]))

    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### check status==0 got all traded
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### check status==1 got partial traded
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### check if any cancellation within 1 sec
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)
    
    # 1. market orders
    readPath = r'L:\orderLog\mdData'
    rawMsgDataSZ = pd.read_pickle(os.path.join(readPath, 'mdLog_msg_%s.pkl'%thisDate))
    orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "agg", "TransactTime"]].reset_index(drop=True)
    orderDataSZ['updateType'] = 0
    tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                             rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
    tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
    tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
    tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
    tradeDataSZ['updateType'] = 4
    cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
    cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
    cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
    cancelDataSZ['updateType'] = 3

    msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
    del orderDataSZ
    del tradeDataSZ
    del cancelDataSZ
    msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
    msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
    msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
    msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
    msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
    msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
    assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
    msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
    msgDataSZ['date'] = int(thisDate) 
    msgDataSZ1 = msgDataSZ
    print('finish market orders')
 

    # 2. orderLog
    infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog['isMsg'] == 1) 
                        & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
    infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
    infoData['Price'] = infoData['Price'].astype('int64')*100
    infoData['OrderQty'] = infoData['absOrderSize']
    infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
    infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
    infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
    statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
    statusInfo.columns = ['order', 'statusLs']
    tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['order', 'TradePriceLs']
    tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['order', 'TradeQtyLs']
    infoData = infoData[infoData['updateType'] == 0]
    infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
    infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])['colo_account'].transform('count')
    display(infoData[infoData['brokerNum'] >= 2].groupby(['colo', 'accCode'])['date'].count())
    display('%.2f%%'%(infoData[infoData['brokerNum'] >= 2].shape[0] / infoData.shape[0]*100))
    display(infoData[infoData['brokerNum'] >= 2].shape[0])
    display(infoData.shape[0])
    infoData = infoData[infoData['brokerNum'] == 1]
    infoData = infoData[['date', 'secid', 'vai', 'ars', 'exchange', 'group', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum', 'order', 'colo', 'accCode']]   
    
    print('finish our orders')
    
    # 3. find the position of our orders
    checkLog = msgDataSZ[msgDataSZ['updateType'] == 0]
    checkLog = pd.merge(checkLog, infoData.drop_duplicates(subset=['secid', 'ApplSeqNum'])[['secid', 'ApplSeqNum', 'ars']], 
                    on=['secid', 'ApplSeqNum'], how='left')
    checkLog['ApplSeqNum'] = np.where(~checkLog['ars'].isnull(), checkLog['ApplSeqNum'], checkLog['ars'])
    checkLog['start_time'] = np.where(~checkLog['ars'].isnull(), checkLog['clockAtArrival'], checkLog['ars'])
    checkLog.drop(["ars"],axis=1,inplace=True)
    checkLog['ApplSeqNum'] = checkLog.groupby(['secid'])['ApplSeqNum'].ffill()
    checkLog['start_time'] = checkLog.groupby(['secid'])['start_time'].ffill()
    checkLog['end_time'] = checkLog['start_time'] + 1*1e6
    checkLog = checkLog[(~checkLog['ApplSeqNum'].isnull()) & (checkLog['clockAtArrival'] > checkLog['start_time']) & 
                       (checkLog['clockAtArrival'] < checkLog['end_time'])]
    msgDataSZ = msgDataSZ[msgDataSZ['StockSeqNum'].isin(checkLog['StockSeqNum'].values)]
    print('finish get the interval')
    
    statusInfo = msgDataSZ.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    statusInfo.columns = ['StockSeqNum', 'statusLs']
    tradePriceInfo = msgDataSZ.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
    tradeQtyInfo = msgDataSZ.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
    checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')  

      
    try:
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], validate='many_to_one')
    except:
        print('There are orders with same pattern but different vai, same ApplSeqNum')
        display([infoData[infoData.duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]])
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])
    savePath = r'L:\orderLog\result\marketPos'
    checkLog.reset_index(drop=True).to_pickle(os.path.join(savePath, 'SZspeed_%s.pkl'%thisDate))
    

'Different orderDirection type:'

array([ 1., -1.])

array([ 1., -1.])

set()

finish market orders


...

colo      accCode
zs_94_04  6237       13
zs_96_08  6282       13
Name: date, dtype: int64

'0.84%'

26

3110

finish our orders
finish get the interval
